In [65]:
import pandas as pd
import numpy as np
import torch

Size (sq ft)	Price (thousands of $)
650	                70
785	                95
1200	            130
1500	            200
1850	            230
2400	            300

In [66]:
x = torch.Tensor([650,785,1200,1500,1850,2400,2800,3000,3200,3200,3400,3600]).unsqueeze(1) 
y = torch.Tensor([70,95,130,200,230,300,340,280, 400,420,500,550]).unsqueeze(1) 

train_split= int(len(x)*0.8)

x_train = x[:train_split]

x_test = x[train_split:]    

y_train = y[:train_split]   

y_test = y[train_split:]

In [67]:
from sklearn.preprocessing import StandardScaler

scaler1 = StandardScaler()
scaler2 = StandardScaler()

scaler1.fit(x_train)
scaler2.fit(y_train)

x_train = torch.tensor(scaler1.transform(x_train), dtype=torch.float32)
x_test = torch.tensor(scaler1.transform(x_test), dtype=torch.float32)
y_train = torch.tensor(scaler2.transform(y_train), dtype=torch.float32)
y_test = torch.tensor(scaler2.transform(y_test), dtype=torch.float32)

In [68]:
from torch import nn

class linear(nn.Module):
    
    def __init__(self) :
        super().__init__()

        self.weights = nn.Parameter(torch.randn(1, requires_grad = True,dtype = torch.float))
        self.bias = nn.Parameter(torch.randn(1, requires_grad = True,dtype = torch.float))

    def forward(self,x):
        return self.weights * x + self.bias

model = linear()

In [69]:
model.state_dict()

OrderedDict([('weights', tensor([-0.6417])), ('bias', tensor([-0.4442]))])

In [70]:
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01,)


epoch = 100

for i in range(epoch):

    model.train()
    #forward pass
    y_pred = model(x_train)

    #calculate the loss function
    loss = loss_function(y_pred, y_train)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    model.eval()

    with torch.inference_mode():

        test_pred = model(x_test)
        test_loss = loss_function(test_pred, y_test)  # Use a different variable to avoid confusion


    if i % 10 == 0:
        print(f"Epoch: {i} | Test Loss: {test_loss.item()} | Loss: {loss.item()}")

Epoch: 0 | Test Loss: 15.405658721923828 | Loss: 2.847578287124634
Epoch: 10 | Test Loss: 11.411028861999512 | Loss: 1.9238660335540771
Epoch: 20 | Test Loss: 8.593353271484375 | Loss: 1.307188868522644
Epoch: 30 | Test Loss: 6.589013576507568 | Loss: 0.8954901099205017
Epoch: 40 | Test Loss: 5.150204658508301 | Loss: 0.6206367015838623
Epoch: 50 | Test Loss: 4.107370376586914 | Loss: 0.4371424615383148
Epoch: 60 | Test Loss: 3.3439419269561768 | Loss: 0.3146401047706604
Epoch: 70 | Test Loss: 2.779343843460083 | Loss: 0.2328566461801529
Epoch: 80 | Test Loss: 2.357534170150757 | Loss: 0.17825736105442047
Epoch: 90 | Test Loss: 2.0392608642578125 | Loss: 0.14180637896060944


In [73]:
new_input = torch.tensor([[2000]], dtype=torch.float32)

new_input_n = torch.tensor(scaler1.transform(new_input),dtype=torch.float32)

model.eval()

with torch.inference_mode():

    y_pred_n = model(new_input_n)

predicted_price = scaler2.inverse_transform(y_pred_n.detach().numpy())
rounded_predicted_price = round(predicted_price[0][0])

print(f"Predicted price for a house with size 2000 sq ft: {rounded_predicted_price} thousands of dollars")
                         



Predicted price for a house with size 2000 sq ft: 227 thousands of dollars


: 

In [48]:
y_pred = model(2000)
y_pred

tensor([2971.2649], grad_fn=<AddBackward0>)

In [46]:
model.state_dict()

OrderedDict([('weights', tensor([1.4862])), ('bias', tensor([-1.0791]))])

## Classification 

In [26]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

# Generate synthetic data
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=42)


In [27]:
import torch 

X = torch.from_numpy(X).type(torch.float) 
y=torch.from_numpy(y).type(torch.float).unsqueeze(1)


In [28]:
X, y[:10]

(tensor([[ 0.6010,  1.5354],
         [ 0.7559, -1.1724],
         [ 1.3545, -0.9485],
         ...,
         [ 2.8443,  0.2113],
         [-0.0259,  1.6193],
         [ 3.6415,  0.7569]]),
 tensor([[1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.],
         [1.]]))

In [29]:
from torch.utils.data import TensorDataset, DataLoader, random_split

dataset = TensorDataset(X, y)

# Print a sample from the dataset to verify
print("First sample in the dataset (input, target):")
print(dataset[0])

# Define the size of the train and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for each subset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


First sample in the dataset (input, target):
(tensor([0.6010, 1.5354]), tensor([1.]))


In [30]:
import torch.nn as nn
import torch.nn.init as init


class classification(nn.Module):

    def __init__(self):
        super().__init__()

        self.layer1 = nn.Linear(in_features =2, out_features =5)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(in_features =5, out_features =5)
        self.relu = nn.ReLU()
        self.layer3 = nn.Linear(in_features =5, out_features =1)
        self.sigmoid = nn.Sigmoid()

       # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        init.kaiming_uniform_(self.layer1.weight, nonlinearity='relu')
        init.kaiming_uniform_(self.layer2.weight, nonlinearity='relu')
        init.xavier_uniform_(self.layer3.weight)

    
    def forward(self,x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        x = self.sigmoid(x)

        return x
        

In [31]:
model = classification()

loss_fn = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


In [32]:
epochs = 100

for i in range(epochs):

    model.train()
    for x_batch, y_batch in train_loader:
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        total_test_loss = 0
        for x_batch_test,y_batch_test in val_loader:
            y_pred_test = model(x_batch_test)
            loss_test = loss_fn(y_pred_test, y_batch_test)
            total_test_loss += loss_test.item()

    if i % 10 == 0:  
        
        print(f"Epoch: {i} | Test Loss: {total_test_loss} |Loss :{loss}")


Epoch: 0 | Test Loss: 4.93992292881012 |Loss :0.6097389459609985
Epoch: 10 | Test Loss: 4.526598036289215 |Loss :0.6433552503585815
Epoch: 20 | Test Loss: 4.210879594087601 |Loss :0.6570175290107727
Epoch: 30 | Test Loss: 3.9151229560375214 |Loss :0.6556727886199951
Epoch: 40 | Test Loss: 3.624013662338257 |Loss :0.43930700421333313
Epoch: 50 | Test Loss: 3.373994380235672 |Loss :0.3710995316505432
Epoch: 60 | Test Loss: 3.1601013839244843 |Loss :0.4291967749595642
Epoch: 70 | Test Loss: 2.9737463295459747 |Loss :0.4223328232765198
Epoch: 80 | Test Loss: 2.809806853532791 |Loss :0.36991798877716064
Epoch: 90 | Test Loss: 2.6676687002182007 |Loss :0.3254834711551666


In [34]:
model.state_dict()

OrderedDict([('layer1.weight',
              tensor([[-1.1517, -1.0421],
                      [ 0.7741,  1.0733],
                      [-1.3224, -1.4856],
                      [-1.2364, -1.7339],
                      [ 1.2889, -1.4183]])),
             ('layer1.bias',
              tensor([-0.2886,  0.3063,  0.3707, -0.2545, -0.3641])),
             ('layer2.weight',
              tensor([[-1.0165,  0.5927,  0.0496,  0.3243, -0.5718],
                      [-1.0418,  0.8909, -0.2275, -0.7642, -0.2403],
                      [-0.3754, -0.2449,  1.0338, -0.1188,  0.1385],
                      [-0.5173,  0.7788, -0.9623, -0.5964, -0.1182],
                      [ 0.9511,  0.1484,  0.2846, -1.0204,  0.4142]])),
             ('layer2.bias',
              tensor([ 0.2465, -0.2832,  0.7252, -0.1924, -0.5247])),
             ('layer3.weight',
              tensor([[ 1.1338, -0.0162, -0.4390,  0.5082,  0.1538]])),
             ('layer3.bias', tensor([-0.2043]))])